In [5]:
import pandas as pd

# Load the Excel files into pandas dataframes
priority_df = pd.read_excel("data inputs/priority invasive plants and visibility months.xlsx")
bordering_df = pd.read_excel("data inputs/species_outside_mgp_2024_calflora_search_bordering.xlsx")
present_mgp_df = pd.read_excel("data inputs/species_present_at_mgp_2024_calflora_search.xlsx")

# Add a 'Source' column to each dataframe and set initial values to None
priority_df['Source'] = None
bordering_df['Source'] = None
present_mgp_df['Source'] = None

# Merge the dataframes
merged_df = pd.concat([priority_df, bordering_df, present_mgp_df])

# Function to update 'Source' column based on conditions
def update_source(row):
    if row['Source'] is None:
        if not pd.isnull(row['Species']) and row['Species'] in priority_df['Species'].values:
            if not pd.isnull(row['Species']) and row['Species'] in present_mgp_df['Species'].values:
                if not pd.isnull(row['Species']) and row['Species'] in bordering_df['Species'].values:
                    row['Source'] = 'Priority List, at MGP and Bordering MGP'
                else:
                    row['Source'] = 'Priority List, locally contained at MGP'
            elif not pd.isnull(row['Species']) and row['Species'] in bordering_df['Species'].values:
                row['Source'] = 'Priority List, bordering MGP'
            else:
                row['Source'] = 'Priority List, not found at MGP or bordering'
        elif not pd.isnull(row['Species']) and row['Species'] in bordering_df['Species'].values:
            if row['Species'] in present_mgp_df['Species'].values:
                row['Source'] = 'Calflora Search found both Bordering and Inside MGP'
            else:
                row['Source'] = 'Calflora Search Bordering MGP'
        elif not pd.isnull(row['Species']) and row['Species'] in present_mgp_df['Species'].values:
            row['Source'] = 'Calflora Search, locally contained at MGP'
    return row

# Apply the function to update the 'Source' column
merged_df = merged_df.apply(update_source, axis=1)

# Drop duplicate species
merged_df.drop_duplicates(subset='Species', keep='first', inplace=True)

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Save the merged dataframe to a new Excel file
merged_df.to_excel("priority_and_cf_search_species.xlsx", index=False)


c:\Users\henry.inman\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [10]:
import pandas as pd

# Load the merged species list
priority = pd.read_excel("python generated excel files\priority_and_cf_search_species.xlsx")

# Load the table without habitat species and debugging columns
without_habitat_df = pd.read_excel("python generated excel files/without_habitat_species_and_debugging_columns.xlsx")

# Perform an inner join on the 'Species' column
filtered_df = pd.merge(without_habitat_df, priority, on='Species', how='inner')

# Rearrange the columns with "Source" as the second column
columns = filtered_df.columns.tolist()
columns.insert(1, columns.pop(columns.index('Source')))
filtered_df = filtered_df[columns]

# Save the filtered dataframe to a new Excel file
filtered_df.to_excel("filtered_table.xlsx", index=False)